# LCEL

[LCEL](https://python.langchain.com/docs/expression_language/) in official docs \
[tutorial video](https://www.youtube.com/watch?v=9M8x485j_lU) on Youtube


In [1]:
import os
import sys

from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI

sys.path.append(os.path.abspath(os.pardir))
load_dotenv()

True

In [2]:
llm = ChatGoogleGenerativeAI(model='gemini-pro')

In [3]:
from langchain.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template('tell me a joke about {foo}')

In [4]:
chain = prompt | llm

In [5]:
chain.invoke({'foo': 'bears'})

AIMessage(content='Why did the bear join the circus?\n\nBecause he wanted to be a bare-back rider!')

In [6]:
from langchain_core.output_parsers import StrOutputParser

chain = prompt | llm | StrOutputParser()
chain.invoke({'foo': 'bears'})

Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised InternalServerError: 500 An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting.


'Why did the bear get kicked out of the restaurant?\n\nBecause he ate all the honey and left no tips!'

In [8]:
chain = prompt | llm.bind(stop=['\n'])
chain.invoke({'foo': 'bears'})

AIMessage(content='Why did the bear get lost in the woods?')

In [9]:
functions = [{
    'name': 'joke',
    'description': 'A joke',
    'parameters': {
        'type': 'object',
        'properties': {
            'setup': {
                'type': 'string',
                'description': 'The setup for the joke'
            },
            'punchline': {
                'type': 'string',
                'description': 'The punchline for the joke'
            }
        },
        'required': ['setup', 'punchline']
    }
}]

chain = prompt | llm.bind(function_call={'name': 'joke'}, functions=functions)

In [10]:
# openai-functionはgeminiでは使えない
import traceback

try:
    chain.invoke({'foo', 'bears'}, config={})
except TypeError:
    traceback.print_exc()

Traceback (most recent call last):
  File "/var/folders/g9/5c_kkmr56nl0nv3f4nty4mym0000gn/T/ipykernel_99440/4238034826.py", line 5, in <module>
    chain.invoke({'foo', 'bears'}, config={})
  File "/Users/ebarakoji/miniforge3/envs/langchain_gemini/lib/python3.11/site-packages/langchain_core/runnables/base.py", line 2053, in invoke
    input = step.invoke(
            ^^^^^^^^^^^^
  File "/Users/ebarakoji/miniforge3/envs/langchain_gemini/lib/python3.11/site-packages/langchain_core/prompts/base.py", line 112, in invoke
    return self._call_with_config(
           ^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/ebarakoji/miniforge3/envs/langchain_gemini/lib/python3.11/site-packages/langchain_core/runnables/base.py", line 1246, in _call_with_config
    context.run(
  File "/Users/ebarakoji/miniforge3/envs/langchain_gemini/lib/python3.11/site-packages/langchain_core/runnables/config.py", line 326, in call_func_with_variable_args
    return func(input, **kwargs)  # type: ignore[call-arg]
           

### JSON Parser


In [11]:
from langchain_core.pydantic_v1 import BaseModel, Field


class Joke(BaseModel):
    setup: str = Field(description='question to set up a joke')
    punchline: str = Field(description='answer to resolve the joke')

In [12]:
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser

parser = JsonOutputParser(pydantic_object=Joke)

prompt = PromptTemplate(
    template='Tell me a joke about {name}.\n{format_instructions}\n',
    input_variables=['name'],
    partial_variables={'format_instructions': parser.get_format_instructions()},
)

chain = prompt | llm | parser

chain.invoke({'name': 'bear'})

{'setup': 'What do you call a bear with no teeth?',
 'punchline': 'A gummy bear!'}

### Paththroughs and itemgetter


In [15]:
from langchain.vectorstores.chroma import Chroma
from langchain.schema.runnable import RunnablePassthrough
from langchain_google_vertexai.embeddings import VertexAIEmbeddings

In [16]:
vectorestore = Chroma.from_texts(['harrison worked at kensho'], embedding=VertexAIEmbeddings('textembedding-gecko@001'))
retriever = vectorestore.as_retriever()

In [18]:
template = '''Answer the question based only on the following context:
{context}

Question: {question}
'''
prompt = ChatPromptTemplate.from_template(template)

In [21]:
chain = ({'context': retriever, 'question': RunnablePassthrough()} | prompt | llm | StrOutputParser())

In [22]:
chain.invoke('where did harrison work?')

Number of requested results 4 is greater than number of elements in index 2, updating n_results = 2


'Kensho'

In [23]:
from operator import itemgetter

template = '''Answer the question based only on the following context:
{context}

Question: {question}

Answer in the following language: {language}
'''
prompt = ChatPromptTemplate.from_template(template)

chain = {
    'context': itemgetter('question') | retriever,
    'question': itemgetter('question'),
    'language': itemgetter('language')
} | prompt | llm | StrOutputParser()

In [26]:
chain.invoke({'question': 'where did harrison work', 'language': 'japanese'})

Number of requested results 4 is greater than number of elements in index 2, updating n_results = 2


' harrisonはkenshoで働いていました'

### Multiple LLM Chains


Docs: [manipulating inputs & output](https://python.langchain.com/docs/expression_language/how_to/map)

In [27]:
prompt1 = ChatPromptTemplate.from_template('what is the city {person} is from?')
prompt2 = ChatPromptTemplate.from_template('what country is the city {city} in? respond in {language}.')

chain1 = prompt1 | llm | StrOutputParser()
chain2 = ({'city': chain1, 'language': itemgetter('language')} | prompt2 | llm | StrOutputParser())
chain2.invoke({'person': 'obama', 'language': 'japanese'})

'ホノルルはアメリカ合衆国ハワイ州にあります。'

In [34]:
from langchain.schema.runnable import RunnableParallel

prompt1 = ChatPromptTemplate.from_template('generate a random color name.')
prompt2 = ChatPromptTemplate.from_template('what is a fruit of color: {color}')
prompt3 = ChatPromptTemplate.from_template('what is the coutries flag that has the color: {color}')
prompt4 = ChatPromptTemplate.from_template('what is the color of {fruit} and {coutry}')

chain1 = prompt1 | llm | StrOutputParser()
chain2 = (RunnableParallel(color=chain1)
          | {
              'fruit': prompt2 | llm | StrOutputParser(),
              'coutry': prompt3 | llm | StrOutputParser(),
}
    | prompt4)

In [35]:
chain2.invoke({})

Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised InternalServerError: 500 An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting.


ChatPromptValue(messages=[HumanMessage(content='what is the color of Kiwi and Libya')])

### Tools

In [36]:
from langchain.tools.ddg_search import DuckDuckGoSearchRun

In [38]:
search = DuckDuckGoSearchRun()

In [39]:
template = '''turn the following user input into a search query for a search engine: 
{input}
'''
prompt = ChatPromptTemplate.from_template(template)

In [40]:
chain = prompt | llm | StrOutputParser() | search

In [42]:
chain.invoke({'input': "I'd like to figure out what games are on tonight."})

"TIME ET. TV. Indiana at Boston. 7:30pm. TNT. The Boston Celtics leads all time series 125-97 versus Indiana Pacers. Last season the Celtics lead regular season series 2-1 versus the Pacers. Boston is 67-28 all time at home versus Indiana. The Celtics and Pacers 2023-24 regular season series is tied 2-2. Sports Games Today has established itself as a leading and trusted online platform for up-to-date live sports TV schedules, ensuring that fans are well-informed and engaged with their favorite sports. Our comprehensive schedules for today cover a wide array of sports, including football, basketball, baseball, hockey, motorsports, soccer, and more. Tonight features a men's Big Ten basketball match up between Purdue vs Illinois streaming on Peacock. The Boilermakers have taken the last four wins against the Fighting Illini including their last game on January 5. Both teams enter tonight on a 3-game win streak. Pre-game coverage begins at 6:30 pm ET. 10:00 AM EST. 2024 NFL Scouting Combin